In [1]:
import findspark
findspark.init("/opt/manual/spark")
from pyspark.sql import SparkSession, functions as F

In [2]:
spark = (
    SparkSession.builder
    .appName("Shuffle Partitions")
    .master("yarn")
    .enableHiveSupport()
    .getOrCreate())

# Read Data

In [3]:
market5 = spark.read.format("parquet") \
.load("/user/train/datasets/market5mil_parquet") \
.orderBy(F.rand())

In [4]:
market5.limit(3).toPandas()

,LOGICALREF,COUNT_,ITEMCODE,ITEMNAME,FICHENO,DATE_,AMOUNT,PRICE,LINENETTOTAL,LINENET,...,CLIENTNAME,BRANDCODE,BRAND,CATEGORY_NAME1,CATEGORY_NAME2,CATEGORY_NAME3,STARTDATE,ENDDATE,SPECODE,CAPIBLOCK_CREADEDDATE
0,1975263,1,00000013899,NESTLE NESGUIK SUT 6 AL 5 ODE,0000000000475654,2017-09-01,1,6.45,6.45,5.97,...,Öznur TÜZÜNER,97,NESTLE,SÜT KAHVALTILIK,SÜT,MEYVELİ SÜT,2017-09-02 18:36:20,2017-09-02 18:37:06,K,2018-07-14 01:56:36
1,1847914,1,00000022628,ULKER HANIMELLER 110 GR.118-,0000000000445169,2017-08-21,1,1.00,1.00,0.93,...,Ömer Faruk ÖZÇALIK,146,ÜLKER,GIDA,BÜSKİVİ ÇEREZ,BÜSKİVİ,2017-08-22 14:20:35,2017-08-22 14:20:46,E,2018-07-14 02:06:57
2,723856,1,00000010520,LIPTON EARL GREY TEA 48 AD 163 GR,0000000000184584,2017-04-16,1,5.50,5.50,5.09,...,Satı ALTINOK,88,LİPTON,İÇECEK,ÇAY KAHVE,ÇAY,2017-04-17 17:00:00,2017-04-17 17:00:40,K,2018-07-14 02:01:01


In [5]:
spark.conf.get("spark.sql.shuffle.partitions")

'200'

# Execute with 200

In [6]:
import time 
start_time = time.time()

market5.groupBy("CITY", "BRANCH").agg(F.sum(F.col("LINENETTOTAL")).alias("Total")) \
.orderBy(F.desc("Total")) \
.limit(20) \
.toPandas()

print("--- %s seconds ---" %(time.time()- start_time))

--- 76.83514618873596 seconds ---


# Execute with 8

In [8]:
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.get("spark.sql.shuffle.partitions")

'8'

In [9]:
import time 
start_time = time.time()

market5.groupBy("CITY", "BRANCH").agg(F.sum(F.col("LINENETTOTAL")).alias("Total")) \
.orderBy(F.desc("Total")) \
.limit(20) \
.toPandas()

print("--- %s seconds ---" %(time.time()- start_time))

--- 12.055027961730957 seconds ---


In [10]:
spark.stop()